In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.stats.proportion import proportion_confint

In [ ]:
df = pd.read_csv("~/Telehealth/Data/telehealth_final_video_230101.csv")

In [ ]:
df = df[df["encounter_status_name"]!= "Canceled"]

In [ ]:
df["encounter_status_name"].value_counts()

In [ ]:
df = df[df["race_ethnic_cat3"]!= "Other"]
df["race_ethnic_cat3"].value_counts()

In [ ]:
def my_recode(is_telehealth):
    if is_telehealth == True:
        return "Telehealth"
    else:
        return "In-person"

In [ ]:
df['is_telehealth'] = df['is_telehealth'].apply(my_recode)
df["is_telehealth"].value_counts()

In [ ]:
df["month_quarter"].value_counts()

In [ ]:
def my_recode(month_quarter):
    if month_quarter == "Jan to Mar 2019":
        return "2019 0103"
    elif month_quarter == "Jan to Mar 2020":
        return "2020 0103"
    elif month_quarter == "Jan to Mar 2021":
        return "2021 0103"
    elif month_quarter == "Jan to Mar 2022":
        return "2022 0103"
    elif month_quarter == "Apr to Jun 2019":
        return "2019 0406"
    elif month_quarter == "Apr to Jun 2020":
        return "2020 0406"
    elif month_quarter == "Apr to Jun 2021":
        return "2021 0406"
    elif month_quarter == "Apr to Jun 2022":
        return "2022 0406"
    elif month_quarter == "Jul to Sep 2019":
        return "2019 0709"
    elif month_quarter == "Jul to Sep 2020":
        return "2020 0709"
    elif month_quarter == "Jul to Sep 2021":
        return "2021 0709"
    elif month_quarter == "Jul to Sep 2022":
        return "2022 0709"
    elif month_quarter == "Oct to Dec 2019":
        return "2019 1012"
    elif month_quarter == "Oct to Dec 2020":
        return "2020 1012"
    elif month_quarter == "Oct to Dec 2021":
        return "2021 1012"
    elif month_quarter == "Oct to Dec 2022":
        return "2022 1012"
    else:
        return 

In [ ]:
df['month_quarter'] = df['month_quarter'].apply(my_recode)
df["month_quarter"].value_counts()

In [ ]:
df_c = df[df["encounter_status_name"]=="Completed"]
df_ns = df[df["encounter_status_name"]=="No Show"]
df_cancel = df[df["encounter_status_name"]=="Canceled"]

In [ ]:
df['insurance_cat2'].value_counts()

In [ ]:
df['insurance_cat2'].mask(df['insurance_cat2'] == 'Medicare Advantage', "Medicare", inplace=True)
df["insurance_cat2"].value_counts()

In [ ]:
df_Mental = df[df["provider_specialty_cat"]=="Mental Health and Behavioral Health"]
df_Endocrinology = df[df["provider_specialty_cat"]=="Endocrinology"]
df_Genetics = df[df["provider_specialty_cat"]=="Genetics"]

df_telehealth = df[df["is_telehealth"]=="Telehealth"]
df_Mental_telehealth = df_telehealth[df_telehealth["provider_specialty_cat"]=="Mental Health and Behavioral Health"]
df_Endocrinology_telehealth = df_telehealth[df_telehealth["provider_specialty_cat"]=="Endocrinology"]
df_Genetics_telehealth = df_telehealth[df_telehealth["provider_specialty_cat"]=="Genetics"]

df_inperson = df[df["is_telehealth"]=="In-person"]
df_Mental_inperson = df_inperson[df_inperson["provider_specialty_cat"]=="Mental Health and Behavioral Health"]
df_Endocrinology_inperson = df_inperson[df_inperson["provider_specialty_cat"]=="Endocrinology"]
df_Genetics_inperson = df_inperson[df_inperson["provider_specialty_cat"]=="Genetics"]

In [ ]:
#both telehealth and inperson
#ALL
data_crosstab = pd.crosstab([df["month_quarter"], df["insurance_cat2"]], 
                             df["encounter_status_name"], margins = False)
total = pd.crosstab([df["month_quarter"], df["insurance_cat2"]], 
                             df["encounter_status_name"]).apply(lambda row: round(row.sum(),1), axis=1).reset_index()
percentages = pd.crosstab([df["month_quarter"], df["insurance_cat2"]], 
                             df["encounter_status_name"]).apply(lambda row: round(row/row.sum()*100,1), axis=1).reset_index()

summaryTable_all=pd.merge(data_crosstab,total,on=["month_quarter","insurance_cat2"])
summaryTable_all=pd.merge(summaryTable_all,percentages,on=["month_quarter","insurance_cat2"])

summaryTable_all["95confint"] = summaryTable_all.apply(lambda x: proportion_confint(x["No Show_x"], x[0], alpha=0.05, method='normal')
                                               ,axis=1)
summaryTable_all['95confint'] = summaryTable_all['95confint'].astype(str)
summaryTable_all['lower_ci'] = summaryTable_all['95confint'].str.split(',').str[0]
summaryTable_all['lower_ci'] = summaryTable_all['lower_ci'].str.replace("(", "")
summaryTable_all['lower_ci'] = summaryTable_all.apply(lambda x: float(x['lower_ci'][0:6])*100,axis=1)

summaryTable_all['upper_ci'] = summaryTable_all['95confint'].str.split(',').str[1]
summaryTable_all['upper_ci'] = summaryTable_all['upper_ci'].str.replace(")", "")
summaryTable_all['upper_ci'] = summaryTable_all.apply(lambda x: float(x['upper_ci'][0:6])*100,axis=1)

#mental health 
#both telehealth and inperson
#ALL
data_crosstab = pd.crosstab([df_Mental["month_quarter"], df_Mental["insurance_cat2"]], 
                             df_Mental["encounter_status_name"], margins = False)
total = pd.crosstab([df_Mental["month_quarter"], df_Mental["insurance_cat2"]], 
                             df_Mental["encounter_status_name"]).apply(lambda row: round(row.sum(),1), axis=1).reset_index()
percentages = pd.crosstab([df_Mental["month_quarter"], df_Mental["insurance_cat2"]], 
                             df_Mental["encounter_status_name"]).apply(lambda row: round(row/row.sum()*100,1), axis=1).reset_index()

summaryTable_Mental=pd.merge(data_crosstab,total,on=["month_quarter","insurance_cat2"])
summaryTable_Mental=pd.merge(summaryTable_Mental,percentages,on=["month_quarter","insurance_cat2"])

summaryTable_Mental["95confint"] = summaryTable_Mental.apply(lambda x: proportion_confint(x["No Show_x"], x[0], alpha=0.05, method='normal')
                                               ,axis=1)
summaryTable_Mental['95confint'] = summaryTable_Mental['95confint'].astype(str)
summaryTable_Mental['lower_ci'] = summaryTable_Mental['95confint'].str.split(',').str[0]
summaryTable_Mental['lower_ci'] = summaryTable_Mental['lower_ci'].str.replace("(", "")
summaryTable_Mental['lower_ci'] = summaryTable_Mental.apply(lambda x: float(x['lower_ci'][0:6])*100,axis=1)

summaryTable_Mental['upper_ci'] = summaryTable_Mental['95confint'].str.split(',').str[1]
summaryTable_Mental['upper_ci'] = summaryTable_Mental['upper_ci'].str.replace(")", "")
summaryTable_Mental['upper_ci'] = summaryTable_Mental.apply(lambda x: float(x['upper_ci'][0:6])*100,axis=1)

In [ ]:
#telehealth

#ALL
data_crosstab = pd.crosstab([df_telehealth["month_quarter"], df_telehealth["insurance_cat2"]], 
                             df_telehealth["encounter_status_name"], margins = False)
total = pd.crosstab([df_telehealth["month_quarter"], df_telehealth["insurance_cat2"]], 
                             df_telehealth["encounter_status_name"]).apply(lambda row: round(row.sum(),1), axis=1).reset_index()
percentages = pd.crosstab([df_telehealth["month_quarter"], df["insurance_cat2"]], 
                             df_telehealth["encounter_status_name"]).apply(lambda row: round(row/row.sum()*100,1), axis=1).reset_index()

summaryTable_telehealth=pd.merge(data_crosstab,total,on=["month_quarter","insurance_cat2"])
summaryTable_telehealth=pd.merge(summaryTable_telehealth,percentages,on=["month_quarter","insurance_cat2"])

summaryTable_telehealth["95confint"] = summaryTable_telehealth.apply(lambda x: proportion_confint(x["No Show_x"], x[0], alpha=0.05, method='normal')
                                               ,axis=1)
summaryTable_telehealth['95confint'] = summaryTable_telehealth['95confint'].astype(str)
summaryTable_telehealth['lower_ci'] = summaryTable_telehealth['95confint'].str.split(',').str[0]
summaryTable_telehealth['lower_ci'] = summaryTable_telehealth['lower_ci'].str.replace("(", "")
summaryTable_telehealth['lower_ci'] = summaryTable_telehealth.apply(lambda x: float(x['lower_ci'][0:6])*100,axis=1)

summaryTable_telehealth['upper_ci'] = summaryTable_telehealth['95confint'].str.split(',').str[1]
summaryTable_telehealth['upper_ci'] = summaryTable_telehealth['upper_ci'].str.replace(")", "")
summaryTable_telehealth['upper_ci'] = summaryTable_telehealth.apply(lambda x: float(x['upper_ci'][0:6])*100,axis=1)

#mental health 
#both telehealth and inperson
#ALL
data_crosstab = pd.crosstab([df_Mental_telehealth["month_quarter"], df_Mental_telehealth["insurance_cat2"]], 
                             df_Mental_telehealth["encounter_status_name"], margins = False)
total = pd.crosstab([df_Mental_telehealth["month_quarter"], df_Mental_telehealth["insurance_cat2"]], 
                             df_Mental_telehealth["encounter_status_name"]).apply(lambda row: round(row.sum(),1), axis=1).reset_index()
percentages = pd.crosstab([df_Mental_telehealth["month_quarter"], df_Mental_telehealth["insurance_cat2"]], 
                             df_Mental_telehealth["encounter_status_name"]).apply(lambda row: round(row/row.sum()*100,1), axis=1).reset_index()

summaryTable_Mental_telehealth=pd.merge(data_crosstab,total,on=["month_quarter","insurance_cat2"])
summaryTable_Mental_telehealth=pd.merge(summaryTable_Mental_telehealth,percentages,on=["month_quarter","insurance_cat2"])

summaryTable_Mental_telehealth["95confint"] = summaryTable_Mental_telehealth.apply(lambda x: proportion_confint(x["No Show_x"], x[0], alpha=0.05, method='normal')
                                               ,axis=1)
summaryTable_Mental_telehealth['95confint'] = summaryTable_Mental_telehealth['95confint'].astype(str)
summaryTable_Mental_telehealth['lower_ci'] = summaryTable_Mental_telehealth['95confint'].str.split(',').str[0]
summaryTable_Mental_telehealth['lower_ci'] = summaryTable_Mental_telehealth['lower_ci'].str.replace("(", "")
summaryTable_Mental_telehealth['lower_ci'] = summaryTable_Mental_telehealth.apply(lambda x: float(x['lower_ci'][0:6])*100,axis=1)

summaryTable_Mental_telehealth['upper_ci'] = summaryTable_Mental_telehealth['95confint'].str.split(',').str[1]
summaryTable_Mental_telehealth['upper_ci'] = summaryTable_Mental_telehealth['upper_ci'].str.replace(")", "")
summaryTable_Mental_telehealth['upper_ci'] = summaryTable_Mental_telehealth.apply(lambda x: float(x['upper_ci'][0:6])*100,axis=1)



In [ ]:
#inperson

#ALL
data_crosstab = pd.crosstab([df_inperson["month_quarter"], df_inperson["insurance_cat2"]], 
                             df_inperson["encounter_status_name"], margins = False)
total = pd.crosstab([df_inperson["month_quarter"], df_inperson["insurance_cat2"]], 
                             df_inperson["encounter_status_name"]).apply(lambda row: round(row.sum(),1), axis=1).reset_index()
percentages = pd.crosstab([df_inperson["month_quarter"], df_inperson["insurance_cat2"]], 
                             df_inperson["encounter_status_name"]).apply(lambda row: round(row/row.sum()*100,1), axis=1).reset_index()

summaryTable_inperson=pd.merge(data_crosstab,total,on=["month_quarter","insurance_cat2"])
summaryTable_inperson=pd.merge(summaryTable_inperson,percentages,on=["month_quarter","insurance_cat2"])

summaryTable_inperson["95confint"] = summaryTable_inperson.apply(lambda x: proportion_confint(x["No Show_x"], x[0], alpha=0.05, method='normal')
                                               ,axis=1)
summaryTable_inperson['95confint'] = summaryTable_inperson['95confint'].astype(str)
summaryTable_inperson['lower_ci'] = summaryTable_inperson['95confint'].str.split(',').str[0]
summaryTable_inperson['lower_ci'] = summaryTable_inperson['lower_ci'].str.replace("(", "")
summaryTable_inperson['lower_ci'] = summaryTable_inperson.apply(lambda x: float(x['lower_ci'][0:6])*100,axis=1)

summaryTable_inperson['upper_ci'] = summaryTable_inperson['95confint'].str.split(',').str[1]
summaryTable_inperson['upper_ci'] = summaryTable_inperson['upper_ci'].str.replace(")", "")
summaryTable_inperson['upper_ci'] = summaryTable_inperson.apply(lambda x: float(x['upper_ci'][0:6])*100,axis=1)

#mental health
data_crosstab = pd.crosstab([df_Mental_inperson["month_quarter"], df_Mental_inperson["insurance_cat2"]], 
                             df_Mental_inperson["encounter_status_name"], margins = False)
total = pd.crosstab([df_Mental_inperson["month_quarter"], df_Mental_inperson["insurance_cat2"]], 
                             df_Mental_inperson["encounter_status_name"]).apply(lambda row: round(row.sum(),1), axis=1).reset_index()
percentages = pd.crosstab([df_Mental_inperson["month_quarter"], df_Mental_inperson["insurance_cat2"]], 
                             df_Mental_inperson["encounter_status_name"]).apply(lambda row: round(row/row.sum()*100,1), axis=1).reset_index()

summaryTable_Mental_inperson=pd.merge(data_crosstab,total,on=["month_quarter","insurance_cat2"])
summaryTable_Mental_inperson=pd.merge(summaryTable_Mental_inperson,percentages,on=["month_quarter","insurance_cat2"])

summaryTable_Mental_inperson["95confint"] = summaryTable_Mental_inperson.apply(lambda x: proportion_confint(x["No Show_x"], x[0], alpha=0.05, method='normal')
                                               ,axis=1)
summaryTable_Mental_inperson['95confint'] = summaryTable_Mental_inperson['95confint'].astype(str)
summaryTable_Mental_inperson['lower_ci'] = summaryTable_Mental_inperson['95confint'].str.split(',').str[0]
summaryTable_Mental_inperson['lower_ci'] = summaryTable_Mental_inperson['lower_ci'].str.replace("(", "")
summaryTable_Mental_inperson['lower_ci'] = summaryTable_Mental_inperson.apply(lambda x: float(x['lower_ci'][0:6])*100,axis=1)

summaryTable_Mental_inperson['upper_ci'] = summaryTable_Mental_inperson['95confint'].str.split(',').str[1]
summaryTable_Mental_inperson['upper_ci'] = summaryTable_Mental_inperson['upper_ci'].str.replace(")", "")
summaryTable_Mental_inperson['upper_ci'] = summaryTable_Mental_inperson.apply(lambda x: float(x['upper_ci'][0:6])*100,axis=1)

In [ ]:
df["insurance_cat2"].value_counts()

In [ ]:
#ALL
summaryTable_commercial_all = summaryTable_all[summaryTable_all["insurance_cat2"]=="Commercial"]
summaryTable_medicare_all = summaryTable_all[summaryTable_all["insurance_cat2"]=="Medicare"]
summaryTable_medicareAdv_all = summaryTable_all[summaryTable_all["insurance_cat2"]=="Medicare Advantage"]
summaryTable_medicaid_all = summaryTable_all[summaryTable_all["insurance_cat2"]=="Medicaid"]
summaryTable_selfpay_all = summaryTable_all[summaryTable_all["insurance_cat2"]=="Self-Pay"]

summaryTable_commercial_telehealth = summaryTable_telehealth[summaryTable_telehealth["insurance_cat2"]=="Commercial"]
summaryTable_medicare_telehealth = summaryTable_telehealth[summaryTable_telehealth["insurance_cat2"]=="Medicare"]
summaryTable_medicareAdv_telehealth = summaryTable_telehealth[summaryTable_telehealth["insurance_cat2"]=="Medicare Advantage"]
summaryTable_medicaid_telehealth = summaryTable_telehealth[summaryTable_telehealth["insurance_cat2"]=="Medicaid"]
summaryTable_selfpay_telehealth = summaryTable_telehealth[summaryTable_telehealth["insurance_cat2"]=="Self-Pay"]

summaryTable_commercial_inperson = summaryTable_inperson[summaryTable_inperson["insurance_cat2"]=="Commercial"]
summaryTable_medicare_inperson = summaryTable_inperson[summaryTable_inperson["insurance_cat2"]=="Medicare"]
summaryTable_medicareAdv_inperson = summaryTable_inperson[summaryTable_inperson["insurance_cat2"]=="Medicare Advantage"]
summaryTable_medicaid_inperson = summaryTable_inperson[summaryTable_inperson["insurance_cat2"]=="Medicaid"]
summaryTable_selfpay_inperson = summaryTable_inperson[summaryTable_inperson["insurance_cat2"]=="Self-Pay"]

#Mental health 
summaryTable_commercial_Mental = summaryTable_Mental[summaryTable_Mental["insurance_cat2"]=="Commercial"]
summaryTable_medicare_Mental = summaryTable_Mental[summaryTable_Mental["insurance_cat2"]=="Medicare"]
summaryTable_medicareAdv_Mental = summaryTable_Mental[summaryTable_Mental["insurance_cat2"]=="Medicare Advantage"]
summaryTable_medicaid_Mental = summaryTable_Mental[summaryTable_Mental["insurance_cat2"]=="Medicaid"]
summaryTable_selfpay_Mental = summaryTable_Mental[summaryTable_Mental["insurance_cat2"]=="Self-Pay"]

summaryTable_commercial_Mental_telehealth = summaryTable_Mental_telehealth[summaryTable_Mental_telehealth["insurance_cat2"]=="Commercial"]
summaryTable_medicare_Mental_telehealth = summaryTable_Mental_telehealth[summaryTable_Mental_telehealth["insurance_cat2"]=="Medicare"]
summaryTable_medicareAdv_Mental_telehealth = summaryTable_Mental_telehealth[summaryTable_Mental_telehealth["insurance_cat2"]=="Medicare Advantage"]
summaryTable_medicaid_Mental_telehealth = summaryTable_Mental_telehealth[summaryTable_Mental_telehealth["insurance_cat2"]=="Medicaid"]
summaryTable_selfpay_Mental_telehealth = summaryTable_Mental_telehealth[summaryTable_Mental_telehealth["insurance_cat2"]=="Self-Pay"]

summaryTable_commercial_Mental_inperson = summaryTable_Mental_inperson[summaryTable_Mental_inperson["insurance_cat2"]=="Commercial"]
summaryTable_medicare_Mental_inperson = summaryTable_Mental_inperson[summaryTable_Mental_inperson["insurance_cat2"]=="Medicare"]
summaryTable_medicareAdv_Mental_inperson = summaryTable_Mental_inperson[summaryTable_Mental_inperson["insurance_cat2"]=="Medicare Advantage"]
summaryTable_medicaid_Mental_inperson = summaryTable_Mental_inperson[summaryTable_Mental_inperson["insurance_cat2"]=="Medicaid"]
summaryTable_selfpay_Mental_inperson = summaryTable_Mental_inperson[summaryTable_Mental_inperson["insurance_cat2"]=="Self-Pay"]


In [ ]:
summaryTable_selfpay_all 

In [ ]:
summaryTable_commercial_telehealth

In [ ]:
summaryTable_selfpay_inperson

In [ ]:
summaryTable_commercial_telehealth

In [ ]:
summaryTable_selfpay_telehealth

In [ ]:
# Creating plot with dataset_1
fig, ((ax1, ax2, ax3),
     (ax4,ax5,ax6)) = plt.subplots(2, 3)

#No Show ALL ----------------------------------------------------------------------------------------
ax1.set_xlabel('Time')
ax1.set_ylabel('Proportion of no-show encounters (%)')
lns1_1 = ax1.plot("month_quarter","No Show_y", 
         data=summaryTable_commercial_all, 
         color = 'tab:orange',
         label="Commercial"
        )
lns1_2 = ax1.plot("month_quarter",
         "No Show_y", 
         data=summaryTable_medicare_all, 
         color = 'm',
         label="Medicare")
lns1_3 = ax1.plot("month_quarter",
         "No Show_y", 
         data=summaryTable_medicareAdv_all, 
         color = 'tab:green',
         label="Medicare Advantage")
lns1_4 = ax1.plot("month_quarter",
         "No Show_y", 
         data=summaryTable_medicaid_all, 
         color = 'tab:blue',
         label="Medicaid")
lns1_5 = ax1.plot("month_quarter",
         "No Show_y", 
         data=summaryTable_selfpay_all, 
         color = 'tab:green',
         label="Uninsured")
ax1.tick_params(axis ='y')

ax1.fill_between(
    summaryTable_commercial_all['month_quarter'], 
    summaryTable_commercial_all['lower_ci'], 
    summaryTable_commercial_all['upper_ci'], 
    color='orange',alpha=0.2)

ax1.fill_between(
    summaryTable_medicare_all['month_quarter'], 
    summaryTable_medicare_all['lower_ci'], 
    summaryTable_medicare_all['upper_ci'], 
    color='pink',alpha=0.2)

ax1.fill_between(
    summaryTable_medicareAdv_all['month_quarter'], 
    summaryTable_medicareAdv_all['lower_ci'], 
    summaryTable_medicareAdv_all['upper_ci'], 
    color='green',alpha=0.2)

ax1.fill_between(
    summaryTable_medicaid_all['month_quarter'], 
    summaryTable_medicaid_all['lower_ci'], 
    summaryTable_medicaid_all['upper_ci'], 
    color='blue',alpha=0.2)


ax1.fill_between(
    summaryTable_selfpay_all['month_quarter'], 
    summaryTable_selfpay_all['lower_ci'], 
    summaryTable_selfpay_all['upper_ci'], 
    color='green',alpha=0.2)

ax1.set_title("No-Show Encounters (Telehealth and In-person)")
ax1.set_ylim(-5, 20)

# added these three lines
lns = lns1_1+lns1_2+lns1_4 + lns1_5
labs = [l.get_label() for l in lns]
ax1.legend(lns, labs, loc=0)
# Names of the xticks albels
time_names = ["Jan 2019","Apr 2019","Jul 2019","Oct 2019",
    "Jan 2020","Apr 2020","Jul 2020","Oct 2020",
    "Jan 2021","Apr 2021","Jul 2021","Oct 2021",
             "Jan 2022","Apr 2022",]
ax1.set_xticklabels(time_names, rotation=90) 

#No Show telehealth ----------------------------------------------------------------------------------------
ax2.set_xlabel('Time')
ax2.set_ylabel('Proportion of no-show encounters (%)')
lns2_1 = ax2.plot("month_quarter","No Show_y", 
         data=summaryTable_commercial_telehealth, 
         color = 'tab:orange',
         label="Commercial"
        )
lns2_2 = ax2.plot("month_quarter",
         "No Show_y", 
         data=summaryTable_medicare_telehealth, 
         color = 'm',
         label="Medicare")
lns2_3 = ax2.plot("month_quarter",
         "No Show_y", 
         data=summaryTable_medicareAdv_telehealth, 
         color = 'tab:green',
         label="Medicare Advantage")
lns2_4 = ax2.plot("month_quarter",
         "No Show_y", 
         data=summaryTable_medicaid_telehealth, 
         color = 'tab:blue',
         label="Medicaid")
lns2_5 = ax2.plot("month_quarter",
         "No Show_y", 
         data=summaryTable_selfpay_telehealth, 
         color = 'tab:green',
         label="Uninsured")
ax2.tick_params(axis ='y')

ax2.fill_between(
    summaryTable_commercial_telehealth['month_quarter'], 
    summaryTable_commercial_telehealth['lower_ci'], 
    summaryTable_commercial_telehealth['upper_ci'], 
    color='orange',alpha=0.2)

ax2.fill_between(
    summaryTable_medicare_telehealth['month_quarter'], 
    summaryTable_medicare_telehealth['lower_ci'], 
    summaryTable_medicare_telehealth['upper_ci'], 
    color='pink',alpha=0.2)

ax2.fill_between(
    summaryTable_medicareAdv_telehealth['month_quarter'], 
    summaryTable_medicareAdv_telehealth['lower_ci'], 
    summaryTable_medicareAdv_telehealth['upper_ci'], 
    color='green',alpha=0.2)

ax2.fill_between(
    summaryTable_medicaid_telehealth['month_quarter'], 
    summaryTable_medicaid_telehealth['lower_ci'], 
    summaryTable_medicaid_telehealth['upper_ci'], 
    color='blue',alpha=0.2)


ax2.fill_between(
    summaryTable_selfpay_telehealth['month_quarter'], 
    summaryTable_selfpay_telehealth['lower_ci'], 
    summaryTable_selfpay_telehealth['upper_ci'], 
    color='green',alpha=0.2)

ax2.set_title("No-Show Encounters (Telehealth)")
ax2.set_ylim(-5, 20)

# added these three lines
lns = lns2_1+lns2_2+lns2_4 +lns2_5
labs = [l.get_label() for l in lns]
ax2.legend(lns, labs, loc=0)
# Names of the xticks albels
time_names = ["Jan 2019","Apr 2019","Jul 2019","Oct 2019",
    "Jan 2020","Apr 2020","Jul 2020","Oct 2020",
    "Jan 2021","Apr 2021","Jul 2021","Oct 2021",
             "Jan 2022","Apr 2022",]
ax2.set_xticklabels(time_names, rotation=90) 

#No Show in-person ----------------------------------------------------------------------------------------
ax3.set_xlabel('Time')
ax3.set_ylabel('Proportion of no-show encounters (%)')
lns3_1 = ax3.plot("month_quarter","No Show_y", 
         data=summaryTable_commercial_inperson, 
         color = 'tab:orange',
         label="Commercial"
        )
lns3_2 = ax3.plot("month_quarter",
         "No Show_y", 
         data=summaryTable_medicare_inperson, 
         color = 'm',
         label="Medicare")
lns3_3 = ax3.plot("month_quarter",
         "No Show_y", 
         data=summaryTable_medicareAdv_inperson, 
         color = 'tab:green',
         label="Medciare Advantage")
lns3_4 = ax3.plot("month_quarter",
         "No Show_y", 
         data=summaryTable_medicaid_inperson, 
         color = 'tab:blue',
         label="Medicaid")
lns3_5 = ax3.plot("month_quarter",
         "No Show_y", 
         data=summaryTable_selfpay_inperson, 
         color = 'tab:green',
         label="Uninsured")
ax3.tick_params(axis ='y')

ax3.fill_between(
    summaryTable_commercial_inperson['month_quarter'], 
    summaryTable_commercial_inperson['lower_ci'], 
    summaryTable_commercial_inperson['upper_ci'], 
    color='orange',alpha=0.2)

ax3.fill_between(
    summaryTable_medicare_inperson['month_quarter'], 
    summaryTable_medicare_inperson['lower_ci'], 
    summaryTable_medicare_inperson['upper_ci'], 
    color='pink',alpha=0.2)

ax3.fill_between(
    summaryTable_medicareAdv_inperson['month_quarter'], 
    summaryTable_medicareAdv_inperson['lower_ci'], 
    summaryTable_medicareAdv_inperson['upper_ci'], 
    color='green',alpha=0.2)

ax3.fill_between(
    summaryTable_medicaid_inperson['month_quarter'], 
    summaryTable_medicaid_inperson['lower_ci'], 
    summaryTable_medicaid_inperson['upper_ci'], 
    color='blue',alpha=0.2)


ax3.fill_between(
    summaryTable_selfpay_inperson['month_quarter'], 
    summaryTable_selfpay_inperson['lower_ci'], 
    summaryTable_selfpay_inperson['upper_ci'], 
    color='green',alpha=0.2)


ax3.set_title("No-Show Encounters (In-person)")
ax3.set_ylim(-5, 20)

# added these three lines
lns = lns3_1+lns3_2+lns3_4 + lns3_5
labs = [l.get_label() for l in lns]
ax3.legend(lns, labs, loc=0)
# Names of the xticks albels
time_names = ["Jan 2019","Apr 2019","Jul 2019","Oct 2019",
    "Jan 2020","Apr 2020","Jul 2020","Oct 2020",
    "Jan 2021","Apr 2021","Jul 2021","Oct 2021",
             "Jan 2022","Apr 2022",]
ax3.set_xticklabels(time_names, rotation=90) 

#No Show MENTAL ALL ----------------------------------------------------------------------------------------
ax4.set_xlabel('Time')
ax4.set_ylabel('Proportion of no-show encounters (%)')
lns4_1 = ax4.plot("month_quarter","No Show_y", 
         data=summaryTable_commercial_Mental, 
         color = 'tab:orange',
         label="Commercial"
        )
lns4_2 = ax4.plot("month_quarter",
         "No Show_y", 
         data=summaryTable_medicare_Mental, 
         color = 'm',
         label="Medicare")
lns4_3 = ax4.plot("month_quarter",
         "No Show_y", 
         data=summaryTable_medicareAdv_Mental, 
         color = 'tab:green',
         label="Medicare Advantage")
lns4_4 = ax4.plot("month_quarter",
         "No Show_y", 
         data=summaryTable_medicaid_Mental, 
         color = 'tab:blue',
         label="Medicaid")

ax4.tick_params(axis ='y')

ax4.fill_between(
    summaryTable_commercial_Mental['month_quarter'], 
    summaryTable_commercial_Mental['lower_ci'], 
    summaryTable_commercial_Mental['upper_ci'], 
    color='orange',alpha=0.2)

ax4.fill_between(
    summaryTable_medicare_Mental['month_quarter'], 
    summaryTable_medicare_Mental['lower_ci'], 
    summaryTable_medicare_Mental['upper_ci'], 
    color='pink',alpha=0.2)

ax4.fill_between(
    summaryTable_medicareAdv_Mental['month_quarter'], 
    summaryTable_medicareAdv_Mental['lower_ci'], 
    summaryTable_medicareAdv_Mental['upper_ci'], 
    color='green',alpha=0.2)

ax4.fill_between(
    summaryTable_medicaid_Mental['month_quarter'], 
    summaryTable_medicaid_Mental['lower_ci'], 
    summaryTable_medicaid_Mental['upper_ci'], 
    color='blue',alpha=0.2)


#ax4.fill_between(
#    summaryTable_selfpay_Mental['year_month'], 
#    summaryTable_selfpay_Mental['lower_ci'], 
#    summaryTable_selfpay_Mental['upper_ci'], 
#    color='yellow',alpha=0.2)

ax4.set_title("No-Show Encounters, Mental Health (Telehealth and In-person)")
ax4.set_ylim(-5, 30)

# added these three lines
lns = lns4_1+lns4_2+lns4_4 #+ lns4_5
labs = [l.get_label() for l in lns]
ax4.legend(lns, labs, loc=0)
# Names of the xticks albels
time_names = ["Jan 2019","Apr 2019","Jul 2019","Oct 2019",
    "Jan 2020","Apr 2020","Jul 2020","Oct 2020",
    "Jan 2021","Apr 2021","Jul 2021","Oct 2021",
             "Jan 2022","Apr 2022",]
ax4.set_xticklabels(time_names, rotation=90) 

#No Show telehealth ----------------------------------------------------------------------------------------
ax5.set_xlabel('Time')
ax5.set_ylabel('Proportion of no-show encounters (%)')
lns5_1 = ax5.plot("month_quarter","No Show_y", 
         data=summaryTable_commercial_Mental_telehealth,
         color = 'tab:orange',
         label="Commercial"
        )
lns5_2 = ax5.plot("month_quarter",
         "No Show_y", 
         data=summaryTable_medicare_Mental_telehealth, 
         color = 'm',
         label="Medicare")
lns5_3 = ax5.plot("month_quarter",
         "No Show_y", 
         data=summaryTable_medicareAdv_Mental_telehealth, 
         color = 'tab:green',
         label="Medicare Advantage")
lns5_4 = ax5.plot("month_quarter",
         "No Show_y", 
         data=summaryTable_medicaid_Mental_telehealth, 
         color = 'tab:blue',
         label="Medicaid")
#lns5_5 = ax5.plot("year_month",
#         "No Show_y", 
#         data=summaryTable_selfpay_Mental_telehealth, 
#         color = 'tab:olive',
#         label="Uninsured")
ax5.tick_params(axis ='y')

ax5.fill_between(
    summaryTable_commercial_Mental_telehealth['month_quarter'], 
    summaryTable_commercial_Mental_telehealth['lower_ci'], 
    summaryTable_commercial_Mental_telehealth['upper_ci'], 
    color='orange',alpha=0.2)

ax5.fill_between(
    summaryTable_medicare_Mental_telehealth['month_quarter'], 
    summaryTable_medicare_Mental_telehealth['lower_ci'], 
    summaryTable_medicare_Mental_telehealth['upper_ci'], 
    color='pink',alpha=0.2)

ax5.fill_between(
    summaryTable_medicareAdv_Mental_telehealth['month_quarter'], 
    summaryTable_medicareAdv_Mental_telehealth['lower_ci'], 
    summaryTable_medicareAdv_Mental_telehealth['upper_ci'], 
    color='green',alpha=0.2)

ax5.fill_between(
    summaryTable_medicaid_Mental_telehealth['month_quarter'], 
    summaryTable_medicaid_Mental_telehealth['lower_ci'], 
    summaryTable_medicaid_Mental_telehealth['upper_ci'], 
    color='blue',alpha=0.2)


#ax5.fill_between(
#    summaryTable_selfpay_Mental_telehealth['year_month'], 
#    summaryTable_selfpay_Mental_telehealth['lower_ci'], 
#    summaryTable_selfpay_Mental_telehealth['upper_ci'], 
#    color='yellow',alpha=0.2)

ax5.set_title("No-Show Encounters, Mental Health (Telehealth)")
ax5.set_ylim(-5, 30)

# added these three lines
lns = lns5_1+lns5_2+lns5_4 #+lns5_5
labs = [l.get_label() for l in lns]
ax5.legend(lns, labs, loc=0)
# Names of the xticks albels
time_names = ["Jan 2019","Apr 2019","Jul 2019","Oct 2019",
    "Jan 2020","Apr 2020","Jul 2020","Oct 2020",
    "Jan 2021","Apr 2021","Jul 2021","Oct 2021",
             "Jan 2022","Apr 2022",]
ax5.set_xticklabels(time_names, rotation=90) 

#No Show in-person ----------------------------------------------------------------------------------------
ax6.set_xlabel('Time')
ax6.set_ylabel('Proportion of no-show encounters (%)')
lns6_1 = ax6.plot("month_quarter","No Show_y", 
         data=summaryTable_commercial_Mental_inperson, 
         color = 'tab:orange',
         label="Commercial"
        )
lns6_2 = ax6.plot("month_quarter",
         "No Show_y", 
         data=summaryTable_medicare_Mental_inperson, 
         color = 'm',
         label="Medicare")
lns6_3 = ax6.plot("month_quarter",
         "No Show_y", 
         data=summaryTable_medicareAdv_Mental_inperson, 
         color = 'tab:green',
         label="Medciare Advantage")
lns6_4 = ax6.plot("month_quarter",
         "No Show_y", 
         data=summaryTable_medicaid_Mental_inperson, 
         color = 'tab:blue',
         label="Medicaid")

ax6.tick_params(axis ='y')

ax6.fill_between(
    summaryTable_commercial_Mental_inperson['month_quarter'], 
    summaryTable_commercial_Mental_inperson['lower_ci'], 
    summaryTable_commercial_Mental_inperson['upper_ci'], 
    color='orange',alpha=0.2)

ax6.fill_between(
    summaryTable_medicare_Mental_inperson['month_quarter'], 
    summaryTable_medicare_Mental_inperson['lower_ci'], 
    summaryTable_medicare_Mental_inperson['upper_ci'], 
    color='pink',alpha=0.2)

ax6.fill_between(
    summaryTable_medicareAdv_Mental_inperson['month_quarter'], 
    summaryTable_medicareAdv_Mental_inperson['lower_ci'], 
    summaryTable_medicareAdv_Mental_inperson['upper_ci'], 
    color='green',alpha=0.2)

ax6.fill_between(
    summaryTable_medicaid_Mental_inperson['month_quarter'], 
    summaryTable_medicaid_Mental_inperson['lower_ci'], 
    summaryTable_medicaid_Mental_inperson['upper_ci'], 
    color='blue',alpha=0.2)


ax6.set_title("No-Show Encounters, Mental Health (In-person)")
ax6.set_ylim(-5, 30)

# added these three lines
lns = lns6_1+lns6_2+lns6_4 #+ lns6_5
labs = [l.get_label() for l in lns]
ax6.legend(lns, labs, loc=0)
# Names of the xticks albels
time_names = ["Jan 2019","Apr 2019","Jul 2019","Oct 2019",
    "Jan 2020","Apr 2020","Jul 2020","Oct 2020",
    "Jan 2021","Apr 2021","Jul 2021","Oct 2021",
             "Jan 2022","Apr 2022",]
ax6.set_xticklabels(time_names, rotation=90) 


fig.set_size_inches(16, 11)
fig.tight_layout(pad=3)

In [ ]:
fig.savefig("insurance.svg", format="svg")

In [ ]:
summaryTable_medicaid_Mental_telehealth